###Implementation of algorithm from "Iterative method for computing the Moore–Penrose inverse based on Penrose equations"
https://www.sciencedirect.com/science/article/pii/S0377042710004917

###Method:
A script has been developed to implement the iterative algorithm of the Moore-Penrose Inverse described in the article "Iterative method for computing the Moore–Penrose inverse based on Penrose equations".

In [ ]:
#Function that implements the algorithm from the article.
def It24C1(A, beta, Niter):
  #A = matrix which you want Penrose-Inverse.
  #beta = variable responsible for convergence.
  #Niter = desired number of iterations.
    Xk = beta*np.transpose(A)
    minnormd = np.inf
    for i in range(1, Niter+1):
        Xk1 = (1+beta)*Xk - beta*np.dot(np.dot(Xk, A), Xk)
        normd = np.linalg.norm(Xk1 - Xk)
        if normd < minnormd:
            minnormd = normd
            Xkmin = Xk
        Xk = Xk1
    return Xkmin #returns Penrose-Inverse of A.

This notebook intends to evaluate its accuracy by inputing a matrix A, calculating its Moore-Penrose Pseudo Inverse using the function numpy.linalg.pinv() and comparing with each iterations of the implemented algorithm. For such, the MSE (Mean Squared Error) was used. 

In [ ]:
#Function that computes MSE.
def MSE(Y_true, Y_pred): #takes parameters original matrix and predicted matrix.
    return np.square(np.subtract(Y_true,Y_pred)).mean()

In [ ]:
#Function based on It24C1(A, beta, Niter) that checks the precision of the algorithm.
def MseDetectorIt24C1(A, beta, Niter):
    truepInv = np.linalg.pinv(A) #Compute Pseudo-Inverse using Python function and store it in truepInv.
    MSEreadings = [] #Create a list of MSE readings.

    #Algorithm of the article.
    Xk = beta*np.transpose(A)
    minnormd = np.inf
    for i in range(1, Niter+1):
        Xk1 = (1+beta)*Xk - beta*np.dot(np.dot(Xk, A), Xk)
        normd = np.linalg.norm(Xk1 - Xk)
        if normd < minnormd:
            minnormd = normd
            Xkmin = Xk
        Xk = Xk1
        MSEreadings.append(MSE(truepInv, Xkmin))#After each iteration,
        #compare the pytorch method result (truepInv) with this algorithm result (Xkmin)


    return MSEreadings #return list for plotting.

This notebook also intends to compare the speed of the proposed algorithm with pytorch np.linalg.pinv().

In [ ]:
#Returns two-element list.
#[0] = Time of Python way.
#[1] = Time of iterative algorithm way.
def TimeTesterIt24C1(A, beta, Niter):

    #create list containing the time taken by pytorch and time taken by algorithm.
    timeTaken = []

    startTime = time.time()
    truepInv = np.linalg.pinv(A)
    timeTaken.append(time.time() - startTime) #append time taken by pytorch.

    startTime = time.time()
    #Algorithm of the article.
    Xk = beta*np.transpose(A)
    minnormd = np.inf
    for i in range(1, Niter+1):
        Xk1 = (1+beta)*Xk - beta*np.dot(np.dot(Xk, A), Xk)
        normd = np.linalg.norm(Xk1 - Xk)
        if normd < minnormd:
            minnormd = normd
            Xkmin = Xk
        Xk = Xk1
    timeTaken.append(time.time() - startTime) #append time taken by the algorithm.

    return timeTaken #return list.

#Results

#1) Testing with the Matrix of the Article (5 lines 4 columns matrix):

This matrix below is presented in the reference article ("Iterative method for computing the Moore–Penrose inverse based on Penrose equations"), example 5.1 (Numerical Example).

##1.1) Convergence parameter $\beta$ = 0.075


* 0.0 seconds for Python.
* 0.0087 seconds for Iterative Method (1000 iterations).

![Graph1](https://drive.google.com/uc?export=view&id=1m1I34-Y2rgLYQ9AzkNxo3E-PO2vABFIT)

##1.2) Convergence parameter $\beta$ = 2 (does not converge)


* 0.0 seconds for Python.
* 0.0139 seconds for Iterative Method (1000 iterations).

![Graph2](https://drive.google.com/uc?export=view&id=1LJzm4zoarN-6bFfGZJrYb5BFEZKUXqAr)

#2) Testing with random, bigger matrixes

##2.1) 512 rows $\times$ 1024 columns matrix,  $\beta = 0.075$
This matrix was created using np.random.random((rows, columns)). It's inputs are random.

* 0.4599 seconds for Python.
* 29.1140 seconds for Iterative Method (1000 iterations).

![Graph3](https://drive.google.com/uc?export=view&id=1A3oqtSAWr6JdU71fBr_4ordiNZhyA75q)

As seen in the result, MSE won't go lower than 0.00175, but it reaches this value in the very first iteration.

##2.2) 1024 rows $\times$ 512 columns, $\beta = 0.075$
This matrix was also created using np.random.random((rows, columns)). It's inputs are random.

* 0.3593 seconds for Python.
* 16.7327 seconds for Iterative Method (1000 iterations).
![Graph4](https://drive.google.com/uc?export=view&id=1s_VfoT6bdf_EMUvAslox9uGU1DJxNCC1)

Same thing happens here. MSE won't go lower than 0.00175, but reaches this value in the first iteration.

#3) Testing with less iterations and with lower convergence parameter $\beta = 0.00075$

##3.1)$1024 \times 1024$ matrix, $\beta = 0.00075$


* 1.0866 seconds for Python.
* 0.5877 seconds for Iterative Method (10 iterations).

![Graph4](https://drive.google.com/uc?export=view&id=1tcdX0QvLmdsduK_0-80isjXFs_kKAxg0)

As seen, the algorithm here gave a result in half the time with a very low MSE.

##3.2) $4096 \times 1024$ matrix, $\beta = 0.00075$
* 1.8269 seconds for Python.
* 1.9171 seconds for Iterative Method (10 iterations).

![Graph5](https://drive.google.com/uc?export=view&id=1fOPbElBA0Cl4s_jQJdkrz1QbGNP96ExG)

##3.3) $1024 \times 4096$ matrix, $\beta = 0.00075$
* 2.4595 seconds for Python.
* 5.6906 seconds for Iterative Method (10 iterations).
![Graph6](https://drive.google.com/uc?export=view&id=1SwBVQ9vHujfFluokpRyF3Rj_EX1ECwhk)

As seen, in some cases the Python way is faster then the Iterative method, but in very larger matrixes MSE will start very low in the very first iteration (e-7).

#4) Testing with only one iteration and with lower convergence parameter $\beta = 0.00075$

In all cases below, MSE was under e-7 in the first iteration:

###4.1)$4096 \times 1024$ matrix, $\beta = 0.00075$
* 1.7918 seconds for Python.
* 0.2898 seconds for Iterative Method

###4.1)$1024 \times 4096$ matrix, $\beta = 0.00075$
* 2.5690 seconds for Python.
* 0.6775 seconds for Iterative Method


###4.3)$1048576 \times 2$ matrix, $\beta = 0.00075$
* 0.0478 seconds for Python.
* 0.0483 seconds for Iterative Method

###4.4)$1048576 \times 2$ matrix, $\beta = 0.00075$
Did not calculate both in Python and Iterative Method since it needed to allocate 8TB of variables in a matrix product.

###4.5)$2097152 \times 2$ matrix, $\beta = 0.00075$
* 0.0969 seconds for Python.
* 0.0974 seconds for Iterative Method

###4.6)$2097152 \times 3$ matrix, $\beta = 0.00075$
* 0.1615 seconds for Python.
* 0.1588 seconds for Iterative Method

###4.7)$2097152 \times 4$ matrix, $\beta = 0.00075$
* 0.2419 seconds for Python.
* 0.2023 seconds for Iterative Method

#Conclusion
The presented algorithm returns good MSE results. It isn't great for very small matrixes, since it takes over 300 iterations to get a good MSE, but in bigger matrixes it was possible to get MSE < e-6 in the first iteration.

In big matrixes, it has been noticed similar time to process, but the iterative algorithm is faster in matrixes with more columns.

#Full code for more testing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

def MSE(Y_true, Y_pred):#return mse
    return np.square(np.subtract(Y_true,Y_pred)).mean()


def It24C1(A, beta, Niter):#iterative algorithm for penrose-inverse
    Xk = beta*np.transpose(A)
    minnormd = np.inf
    for i in range(1, Niter+1):
        Xk1 = (1+beta)*Xk - beta*np.dot(np.dot(Xk, A), Xk)
        normd = np.linalg.norm(Xk1 - Xk)
        if normd < minnormd:
            minnormd = normd
            Xkmin = Xk
        Xk = Xk1
    return Xkmin

def MseDetectorIt24C1(A, beta, Niter):#iterative algorithm for penrose-inverse*
#returns list of MSE between python way and iterative algorith way for each iteration

    truepInv = np.linalg.pinv(A)
    MSEreadings = []

    Xk = beta*np.transpose(A)
    minnormd = np.inf
    for i in range(1, Niter+1):
        Xk1 = (1+beta)*Xk - beta*np.dot(np.dot(Xk, A), Xk)
        normd = np.linalg.norm(Xk1 - Xk)
        if normd < minnormd:
            minnormd = normd
            Xkmin = Xk
        Xk = Xk1
        MSEreadings.append(MSE(truepInv, Xkmin))

    return MSEreadings 

def plotIterationsxMSE(A, beta, nOfIterations): #plot iterations

    y = MseDetectorIt24C1(A, beta, nOfIterations)
    x = np.array(range(len(y)))
    x = x + 1


    positionforMiniPlot = 0
    for element in y:
        positionforMiniPlot += 1
        if element < 0.5:
            break 
    plt.margins(x=0)
    plt.xlabel("Iteration Number")
    plt.ylabel("MSE")

    miniplotY = y[positionforMiniPlot:positionforMiniPlot+60]
    miniplotX = np.array(range(len(y))[positionforMiniPlot:positionforMiniPlot+60]) #plots 60 readings after convergence
    plt.scatter(miniplotX, miniplotY)
    plt.savefig("MSE miniplot.png", bbox_inches='tight') #make graph of miniplot
    plt.clf()

    plt.xlabel("Iteration Number")
    plt.ylabel("MSE")
    #print("\nMatrix used: ")
    #print(A)
    plt.margins(x=0)
    plt.plot(x, y)
    plt.savefig("MSE bigplot.png", bbox_inches='tight') #make graph of big plot

def TimeTesterIt24C1(A, beta, Niter):

    #create list containing the time taken by pytorch and time taken by algorithm.
    timeTaken = []

    startTime = time.time()
    truepInv = np.linalg.pinv(A)
    timeTaken.append(time.time() - startTime) #append time taken by pytorch.

    startTime = time.time()
    #Algorithm of the article.
    Xk = beta*np.transpose(A)
    minnormd = np.inf
    for i in range(1, Niter+1):
        Xk1 = (1+beta)*Xk - beta*np.dot(np.dot(Xk, A), Xk)
        normd = np.linalg.norm(Xk1 - Xk)
        if normd < minnormd:
            minnormd = normd
            Xkmin = Xk
        Xk = Xk1
    timeTaken.append(time.time() - startTime) #append time taken by the algorithm.

    return timeTaken #return list.

articleMatrixA = np.array([[0.8846, 0.807516, 0.381614, 0.671798], 
    [0.854492, 0.91836, 0.611953, 0.664359], [0.673669, 0.459477, 0.383368, 0.575746], 
    [0.865487, 0.803065, 0.523343, 0.687009], [1.14976, 0.964402, 0.594889, 0.918999]])
usedMatrix = np.random.random((1048576*2, 4))


print(TimeTesterIt24C1(usedMatrix, 0.00075, 1))
plotIterationsxMSE(usedMatrix, 0.00075, 1)